# WikiGen Agent Workflow Testing

This notebook provides an environment to test the WikiGen agent-based story processing workflow using the new **BaseAgent architecture**. Each agent can be tested individually with different LLM models to demonstrate the flexibility and modularity of the system.

## 🏗️ BaseAgent Architecture

All WikiGen agents now inherit from `BaseAgent`, providing:
- **Default Models**: Each agent has configurable default provider/model
- **Override Flexibility**: Can override provider/model per call when needed  
- **Centralized LLM Logic**: Common error handling and validation
- **Clean API**: Simplified method signatures with optional parameters

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

Each agent demonstrates the BaseAgent pattern with different default models to show architectural flexibility.

## ⚙️ Setup

Make sure the Portkey Gateway is running to use the LLM Service:

```bash
docker run \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

The following cells will:
- use the Portkey Gateway to test the LLM Service
- initialize the LLM Service
- import the WikiGen workflow agents
- load a test story from the `tests/resources/pokemon_amber/story` directory


In [1]:
# Cell 1: Setup and Configuration for LLM Service Testing
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
import logging
from uuid import uuid4

# Set environment to skip database for LLM service testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"  # Default Portkey Gateway

# Add the backend/src directory to sys.path so we can import our modules
# This assumes you are running the notebook from the `backend/` directory or VS Code multi-root
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir() and (notebook_dir / 'pyproject.toml').is_file():
    # This means we're likely in the backend/ directory itself
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir() and (notebook_dir.parent / 'pyproject.toml').is_file():
    # This means we're likely in the backend/notebooks/ directory
    sys.path.insert(0, str(notebook_dir.parent / 'src'))
else:
    print("Warning: Could not automatically add 'src/' to Python path. Please ensure your current directory allows imports from src/")

# Configure logging for better output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
# Reduce noise from third-party loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('uvicorn.access').setLevel(logging.WARNING)
logging.getLogger('shuscribe').setLevel(logging.INFO)

print("🧪 LLM Service Testing Notebook")
print("=" * 50)
print(f"Current working directory: {os.getcwd()}")
print(f"Database mode: {'IN-MEMORY' if os.environ.get('SKIP_DATABASE') == 'true' else 'SUPABASE'}")
print(f"Portkey Gateway: {os.environ.get('PORTKEY_BASE_URL', 'Not configured')}")
print("Autoreload enabled. Changes to .py files in src/ will be reloaded.")
print("\n💡 This notebook tests LLM service functionality without requiring database setup.")
print("   Perfect for testing direct API key usage and LLM provider integrations.")

🧪 LLM Service Testing Notebook
Current working directory: /home/jimnix/gitrepos/shuscribe/backend/notebooks
Database mode: IN-MEMORY
Portkey Gateway: http://localhost:8787/v1
Autoreload enabled. Changes to .py files in src/ will be reloaded.

💡 This notebook tests LLM service functionality without requiring database setup.
   Perfect for testing direct API key usage and LLM provider integrations.


In [2]:
# Cell 2: Import Modules (Updated for Supabase)
from src.config import settings
from src.services.llm.llm_service import LLMService
from src.api.dependencies import get_user_repository_dependency
from src.schemas.llm.models import LLMMessage, LLMResponse
from src.core.encryption import encrypt_api_key  # Import encryption function
from dotenv import dotenv_values
from src.database.models.user import UserCreate, UserAPIKeyCreate

print("✅ Modules imported successfully.")

# Display current settings
print("\n--- Current Settings ---")
print(f"DEBUG: {settings.DEBUG}")
print(f"ENVIRONMENT: {settings.ENVIRONMENT}")
print(f"SKIP_DATABASE: {settings.SKIP_DATABASE}")
print(f"PORTKEY_BASE_URL: {settings.PORTKEY_BASE_URL}")
if not settings.SKIP_DATABASE:
    print(f"SUPABASE_URL: {settings.SUPABASE_URL}")
    print(f"SUPABASE_KEY: {'***' + settings.SUPABASE_KEY[-4:] if len(settings.SUPABASE_KEY) > 4 else 'Not set'}")
else:
    print("DATABASE_MODE: In-Memory (Supabase skipped)")
print("------------------------")

print("\n🔧 Import Summary:")
print("✅ LLMService - Ready for testing")
print("✅ Repository Factory - Automatic in-memory/Supabase switching") 
print("✅ Pydantic Models - Type-safe user schemas")
print("✅ Encryption - API key encryption/decryption")  # Updated
print("✅ Supabase Connection - Available when SKIP_DATABASE=false")

# WikiGen agent imports
from src.agents.wikigen import (
    WikiGenOrchestrator,
    ArcSplitterAgent,
    WikiPlannerAgent,
    ArticleWriterAgent,
    GeneralSummarizerAgent,
    ChapterBacklinkerAgent
)

from src.schemas.llm.models import ThinkingEffort

2025-06-30 23:34:49,426 - src.config - INFO - Pydantic Settings 'extra' mode set to: 'ignore' for environment: 'development'
✅ Modules imported successfully.

--- Current Settings ---
DEBUG: True
ENVIRONMENT: development
SKIP_DATABASE: True
PORTKEY_BASE_URL: http://localhost:8787/v1
DATABASE_MODE: In-Memory (Supabase skipped)
------------------------

🔧 Import Summary:
✅ LLMService - Ready for testing
✅ Repository Factory - Automatic in-memory/Supabase switching
✅ Pydantic Models - Type-safe user schemas
✅ Encryption - API key encryption/decryption
✅ Supabase Connection - Available when SKIP_DATABASE=false


In [3]:
# Cell 3: Initialize LLM Service and Store Encrypted API Keys

print("🚀 Initializing LLM Service...")

# The factory automatically chooses in-memory or Supabase based on SKIP_DATABASE
llm_service = LLMService(user_repository=get_user_repository_dependency())

# Ensure repository is available
if not llm_service.user_repository:
    raise RuntimeError("Failed to initialize user repository")

print("✅ LLM Service initialized successfully!")
print(f"📁 Repository type: {type(llm_service.user_repository).__name__}")
print(f"🛡️  Database mode: {'In-Memory' if settings.SKIP_DATABASE else 'Supabase'}")

if settings.SKIP_DATABASE:
    print("\n💡 Running in database-free mode:")
    print("   • Perfect for testing with direct API keys")
    print("   • No Supabase setup required")
    print("   • User API keys stored in memory only")
else:
    print("\n🗄️  Connected to Supabase:")
    print("   • User API keys stored encrypted in database")
    print("   • Full multi-user support enabled")
    print("   • Row-level security active")

print("\n🔑 Loading and storing encrypted API keys...")

# Load environment variables once
env_values = dotenv_values()

# Create a test user for API key storage
TEST_USER = await llm_service.user_repository.create(UserCreate(
    email="test@example.com",
    display_name="Test User"
))

print(f"👤 Created test user: {TEST_USER.email} (ID: {TEST_USER.id})")

# Store API keys from environment in the repository (properly encrypted)
stored_keys = 0
available_providers = []
for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    
    if api_key:
        # Properly encrypt the API key before storing
        encrypted_key = encrypt_api_key(api_key)
        
        await llm_service.user_repository.store_api_key(
            user_id=TEST_USER.id,
            api_key_data=UserAPIKeyCreate(
                provider=provider_id,
                api_key=encrypted_key,  # Now properly encrypted
                provider_metadata={}
            )
        )
        stored_keys += 1
        available_providers.append(provider.display_name)
        print(f"   🔐 Stored and encrypted {provider.display_name} API key")

print(f"✅ Stored {stored_keys} encrypted API keys in repository")
if available_providers:
    print(f"📋 Available providers: {', '.join(available_providers)}")
print("\n🎯 Ready for testing!")

🚀 Initializing LLM Service...
✅ LLM Service initialized successfully!
📁 Repository type: FileUserRepository
🛡️  Database mode: In-Memory

💡 Running in database-free mode:
   • Perfect for testing with direct API keys
   • No Supabase setup required
   • User API keys stored in memory only

🔑 Loading and storing encrypted API keys...
👤 Created test user: test@example.com (ID: d6946dad-2142-4a3f-98a2-49c613090d56)
   🔐 Stored and encrypted OpenAI API key
   🔐 Stored and encrypted Google API key
   🔐 Stored and encrypted Anthropic API key
✅ Stored 3 encrypted API keys in repository
📋 Available providers: OpenAI, Google, Anthropic

🎯 Ready for testing!


In [4]:
# Cell 4: Load Test Story

print("🚀 Initializing Services and Loading Test Story...")
print("=" * 60)

# Import the StoryImporter class and dependencies
import sys
from pathlib import Path
from uuid import UUID

# Ensure we can import from src/ and scripts/
backend_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(backend_root / "src"))

# Import required modules
from src.utils.test.import_story import StoryImporter
from src.database.models.repositories import Repositories, IStoryRepository
from src.database.factory import get_repositories
from src.database.models.story import FullStoryBase


print("\n📖 Loading Test Story...")
story_directory_path = backend_root / "tests" / "resources" / "pokemon_amber" / "story"

try:
    if not story_directory_path.exists():
        raise FileNotFoundError(f"Pokemon Amber story directory not found: {story_directory_path}")
    
    # Create a temporary workspace for the story
    temp_workspace_path = backend_root / "temp" / "pokemon_amber_notebook"
    temp_workspace_path.mkdir(parents=True, exist_ok=True)
    
    # Initialize the importer and import the story
    importer = StoryImporter(temp_workspace_path)
    import_result = await importer.import_story_from_directory(story_directory_path)
    
    print(f"✅ Loaded: {import_result['story_title']}")
    print(f"   Chapters: {import_result['chapters_imported']}")
    print(f"   Workspace ID: {import_result['workspace_id']}")
    
    # Get the repositories to access the imported data
    STORY_REPO: IStoryRepository = get_repositories(backend="file", workspace_path=temp_workspace_path).story
    WORKSPACE_ID = UUID(import_result["workspace_id"])
    
    # Get story metadata and chapters
    story_metadata = await STORY_REPO.get_story_metadata(WORKSPACE_ID)
    chapters = await STORY_REPO.get_chapters_by_workspace(WORKSPACE_ID)
    
    # Create a FullStoryBase object for compatibility with existing code
    STORY = FullStoryBase(metadata=story_metadata, chapters=chapters)
    
    print(f"   Author: {STORY.metadata.author}")
    print(f"   Genres: {', '.join(STORY.metadata.genres)}")
    
    # Prepare story content for agent testing
    story_content = "\n\n".join([
        f"# Chapter {chapter.chapter_number}: {chapter.title}\n{chapter.content}" 
        for chapter in sorted(STORY.chapters, key=lambda c: c.chapter_number)
    ])
    
    print(f"\n📊 Test Data Summary:")
    print(f"   Total characters: {len(STORY.get_full_content()):,}")
    print(f"   Total word count: {STORY.word_count:,}")
    print(f"   First chapter: {STORY.chapters[0].title}")
    print(f"   Last chapter: {STORY.chapters[-1].title}")
    
except Exception as e:
    print(f"❌ Error loading story: {e}")
    import traceback
    traceback.print_exc()
    raise

🚀 Initializing Services and Loading Test Story...

📖 Loading Test Story...
Importing story from: /home/jimnix/gitrepos/shuscribe/backend/tests/resources/pokemon_amber/story
🔍 Checking metadata_root for chapters...
✅ Found Chapters element in metadata
📋 Found 8 chapter elements (including any in comments)
✅ Successfully loaded 8 chapters from metadata
  📄 Imported: [Chapter 1] Truck-kun Strikes Again
  📄 Imported: [Chapter 2] All Aboard!
  📄 Imported: [Chapter 3] Into the World of (Pocket) Monsters
  📄 Imported: [Chapter 4] Achievement Unlocked! First Battle!
  📄 Imported: [Chapter 5] A Perfectly Normal Gym Session
  📄 Imported: [Chapter 6] Scientist Fuji
  📄 Imported: [Chapter 7] Side Quests Galore
  📄 Imported: [Chapter 8] Start of an Unpaid Side Quest
✅ Successfully imported 'Pokemon: Ambertwo' with 8 chapters
✅ Loaded: Pokemon: Ambertwo
   Chapters: 8
   Workspace ID: bd1b408e-ea0f-4b39-bc41-fe9c2cd6ebf1
   Author: ChronicImmortality
   Genres: Drama, Action, Adventure, Fantasy

📊 T

# Run individual agents

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

## ArcSplitter Agent - Streaming Analysis

**🔄 Single LLM Call with Real-time Streaming!**

The ArcSplitter agent now supports **streaming analysis** that provides real-time feedback while accumulating the final structured result. This approach uses **only one LLM call** for both user experience and final parsing.

- ⚡ **Real-time Feedback** - See analysis progress as it happens
- 🚀 **Single LLM Call** - No wasteful duplicate API calls  
- 📊 **Live Updates** - Stream response chunks as they're generated
- 🎯 **Smart Accumulation** - Parse final accumulated result into structured data
- 💰 **Cost Efficient** - One call gives you both streaming UX and structured output


In [5]:
# Cell: 4 Test ArcSplitter Agent - New Streaming Architecture

print("\n🔄 Testing ArcSplitter Agent - NEW STREAMING ARCHITECTURE")
print("=" * 60)
print("This demonstrates the new clean separation of concerns:")
print("📡 Streaming: Raw LLM responses with proper chunk type labels")
print("🎯 State Management: Internal accumulation and parsing")
print("🔗 Result Access: Clean final result via get_final_result()")

# provider = "google"  # Use Google for streaming demo    
# api_key = AVAILABLE_PROVIDERS[provider]['api_key']
# model = LLMService.get_default_test_model_name_for_provider(provider)
# model = "claude-sonnet-4-20250514"
# model = "gemini-2.5-flash-lite-preview-06-17"
# model = "o4-mini"

# print(f"🔧 Streaming with: {provider} / {model}")

if WORKSPACE_ID is not None:
    STORY_METADATA = await STORY_REPO.get_story_metadata(WORKSPACE_ID)
    print(f"📊 Story: {STORY_METADATA.title} ({STORY_METADATA.total_chapters} chapters)")

try:
    # Initialize the agent
    arc_splitter_streaming = ArcSplitterAgent(
        llm_service=llm_service,
        # default_provider=provider,
        # default_model=model,
        temperature=0.7,
        max_tokens=16000,
        thinking=ThinkingEffort.LOW
    )
    
    print(f"\n⚡ Starting streaming analysis...")
    print("📡 Live stream output (by chunk type):")
    print("-" * 40)
    
    # Import ChunkType for proper enum usage
    from src.schemas.llm.models import ChunkType
    
    # Track streaming responses by type - Using enum constants
    chunk_counts = {ChunkType.THINKING: 0, ChunkType.CONTENT: 0, ChunkType.UNKNOWN: 0}
    total_content_chars = 0
    
    async for chunk in arc_splitter_streaming.analyze_story_streaming(
        story=STORY,
        user_id=TEST_USER.id,
    ):
        # Use the enum directly - no need for .value
        chunk_type = chunk.chunk_type
        chunk_counts[chunk_type] += 1
        
        # Get metadata for debugging
        metadata = chunk.metadata or {}
        chunk_num = metadata.get("chunk_number", "?")
        chapters = metadata.get("chapters", "?")
        
        # Show chunk info - use .value for display
        print(f"[{chunk_type.value.upper()}] Chunk {chunk_num} ({chapters}): {len(chunk.content)} chars")
        
        # For CONTENT chunks, show a preview of the actual content
        if chunk_type == ChunkType.CONTENT and chunk.content:
            total_content_chars += len(chunk.content)
            # Show first bit of content for CONTENT chunks
            preview = chunk.content[:100].replace("\n", "\\n")
            print(f"  📝 Content preview: {preview}...")
        
        # For THINKING chunks, just show that we got thinking
        elif chunk_type == ChunkType.THINKING and chunk.content:
            print(f"  🤔 Thinking: {len(chunk.content)} chars")
    
    print("-" * 40)
    print(f"✅ Streaming completed!")
    print(f"📊 Chunk counts: {chunk_counts}")
    print(f"📝 Total content characters: {total_content_chars}")
    
    # Get internal chunk processing details
    chunk_results = arc_splitter_streaming.get_window_results()
    print(f"\n🔍 Internal Chunk Processing Summary:")
    for chunk_result in chunk_results:
        print(f"  Chunk {chunk_result.window_number}: {chunk_result.chapters_range}")
        raw = chunk_result.raw_content
        print(f"    📝 Accumulated: T={len(raw.thinking)}, C={len(raw.content)}, U={len(raw.unknown)} chars")
        if chunk_result.parsed_result:
            print(f"    ✅ Parsed: {len(chunk_result.parsed_result.arcs)} arcs")
        if chunk_result.error:
            print(f"    ❌ Error: {chunk_result.error}")
    
    # Get the final merged result (no manual parsing needed!)
    print(f"\n🎯 Getting final result...")
    try:
        final_result = arc_splitter_streaming.get_final_result()
        
        print(f"✅ Final result ready!")
        print(f"📖 Total arcs generated: {len(final_result.arcs)}")
        
        # Show all arcs
        for i, arc in enumerate(final_result.arcs):
            print(f"\n🏛️  Arc {i+1}: {arc.title}")
            print(f"   📖 Chapters: {arc.start_chapter}-{arc.end_chapter}")
            print(f"   📝 Summary: {arc.summary[:100]}...")
            print(f"   🔒 Finalized: {arc.is_finalized}")
            
    except Exception as parse_error:
        print(f"❌ Could not get final result: {parse_error}")
        print("💡 Check the chunk processing summary above for errors")
    
except Exception as e:
    print(f"❌ Streaming test failed: {e}")
    import traceback
    traceback.print_exc()


🔄 Testing ArcSplitter Agent - NEW STREAMING ARCHITECTURE
This demonstrates the new clean separation of concerns:
📡 Streaming: Raw LLM responses with proper chunk type labels
🎯 State Management: Internal accumulation and parsing
🔗 Result Access: Clean final result via get_final_result()
📊 Story: Pokemon: Ambertwo (8 chapters)

⚡ Starting streaming analysis...
📡 Live stream output (by chunk type):
----------------------------------------
2025-06-30 23:34:52,555 - src.agents.wikigen.arc_splitter - INFO - 🔄 Starting new analysis: fda56d86-33cd-4a02-99a4-cacc9e0218f3
2025-06-30 23:34:52,556 - src.agents.wikigen.arc_splitter - INFO - 📐 Model Context Window: 1,048,576 tokens
2025-06-30 23:34:52,557 - src.agents.wikigen.arc_splitter - INFO - 📐 Chunk Limit: 1,043,576 tokens (after 5,000 overhead)
2025-06-30 23:34:52,557 - src.agents.wikigen.arc_splitter - INFO - 🔍 ArcSplitter Analysis Starting:
2025-06-30 23:34:52,558 - src.agents.wikigen.arc_splitter - INFO -    📖 Story: Pokemon: Ambertwo
202

2025-06-30 23:34:52,562 - src.agents.wikigen.arc_splitter - INFO -    🧮 Chunk tokens: 27507, Final: True
2025-06-30 23:34:52,565 - src.agents.wikigen.arc_splitter - INFO -    🤖 Making LLM call for window 1...
2025-06-30 23:34:52,567 - src.services.llm.llm_service - INFO - Using database API key for provider=google, model=gemini-2.5-flash-lite-preview-06-17, user=d6946dad-2142-4a3f-98a2-49c613090d56
2025-06-30 23:34:52,628 - src.services.llm.llm_service - INFO - Set strict_open_ai_compliance=False for thinking mode
2025-06-30 23:34:52,630 - src.services.llm.llm_service - INFO - Model gemini-2.5-flash-lite-preview-06-17 supports structured output - using response_format parameter
2025-06-30 23:34:52,630 - src.services.llm.llm_service - INFO - Using simplified schema for Google Gemini (removed validation constraints)
2025-06-30 23:34:52,631 - src.services.llm.llm_service - INFO - Making LLM request: provider=google, model=gemini-2.5-flash-lite-preview-06-17, gateway=http://localhost:8787/

In [7]:
print(chunk_results[0].raw_content.thinking)

**Refining Arc Divisions**

I'm currently focused on the narrative arcs within the first eight chapters of "Pokemon: Ambertwo." My growth-aware, conservative approach is guiding me to identify 3 distinct arcs that encompass the initial story elements. I am also working on clearly defining each arc's focus. My goal is to create building blocks for future expansion.


**Finalizing Arc Boundaries**

I'm now zeroing in on defining the precise borders for the arcs within the complete "Pokemon: Ambertwo" narrative. Considering this is the entire story, the growth-aware approach is emphasizing a focus on broader arcs that reflect the core narrative shifts, ensuring they're substantial and encompassing. I'm actively analyzing the key transitions, theme developments, and character arcs to establish the most impactful divisions. My goal remains building blocks, while also ensuring the arcs are big enough to be useful in the long run.


**Assessing Narrative Shifts**

Now I'm diving deeper into t

In [17]:
print(final_result.model_dump_json(indent=2))

{
  "story_prediction": "The story is poised for a long-term narrative focused on the protagonist's journey of self-discovery and her role in combating Team Rocket's ambitions. Future plotlines will likely involve her mastering her abilities as a trainer, exploring different regions of the Pokemon world, and uncovering the secrets behind Mewtwo and other genetically engineered Pokemon. Her unique meta-knowledge will be a recurring advantage, but the reality of the world will challenge her assumptions. Team Rocket's pursuit of powerful Pokemon and their exploitation of Pokemon biology will drive central conflicts, potentially leading to direct confrontations with their leadership and research divisions. Dr. Fuji's complex role as a scientist, father, and Team Rocket operative will likely lead to moral dilemmas and potential betrayals or alliances. The protagonist's connection to Amber's mother, Delia, may also become a significant plot thread.",
  "growth_assessment": "This story has im

## WikiPlanner Agent

In [22]:
#
#
#

## ArticleWriterAgent

In [23]:
#
#
#

## GeneralSummarizerAgent

In [24]:
#
#
#

## WikiGenOrchestrator

In [25]:
#
#
#